# 6: Motorized IVTT and OVTT

Model 6 relaxes the travel time constraint further than Model 5 by disaggregating the travel time for motorized modes into distinct components for in-vehicle travel time (IVT) and out-of-vehicle travel time (OVT). This specification allows the two components of travel time for motorized travel to have different effects on utility with the expectation that travelers are more sensitive to out-of-vehicle time than in-vehicle time. (pp. 111)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2") + P("hhinc#2,3") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#2,3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

People may value time differently depending on what mode they’re in, so we’ll decompose tottime into ovtt and ivtt for the motorized alternatives.

In [ ]:
m.utility_ca = (
    +P("nonmotorized_time") * X("(altid>4) * tottime")
    + P("motorized_ovtt") * X("(altid <= 4) * ovtt")
    + P("motorized_ivtt") * X("(altid <= 4) * ivtt")
    + PX("totcost")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 6, Motorized Times"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, method="bhhh")
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        "totcost",
        "nonmotorized_time",
        "motorized_ivtt",
        "motorized_ovtt",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_BIKE": -1.719093452086421,
    "ASC_SR2": -2.4299943985819956,
    "ASC_SR3P": -3.8834310483961114,
    "ASC_TRAN": -0.4898195783250097,
    "ASC_WALK": 0.4091981983094832,
    "hhinc#2,3": -0.0015835444926238225,
    "hhinc#4": -0.005692708506297093,
    "hhinc#5": -0.01221964685106249,
    "hhinc#6": -0.009303042887650576,
    "motorized_ivtt": -0.002540641825564394,
    "motorized_ovtt": -0.07594330410291915,
    "nonmotorized_time": -0.06319642667912634,
    "totcost": -0.0047982963933021535,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )